In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import os
import folium
import json

In [2]:
%matplotlib inline


In [3]:
# Import ".json" file for the U.S. 
country_geo = r'C:\Users\Bilel\Desktop\data_analyst_Carrer Foundry\Data Immersion\A6-Advanced Analytics & Dashboard Design\20-01-24 NY Citi Bike project\02 Data\Original Data\new-york-city-boroughs-ny_.geojson'

In [4]:
path= r'C:\Users\Bilel\Desktop\data_analyst_Carrer Foundry\Data Immersion\A6-Advanced Analytics & Dashboard Design\20-01-24 NY Citi Bike project'


In [5]:
# import  datset
df=pd.read_csv(os.path.join(path,'02 Data','Prepared Data','citi_derived.csv'),index_col = False)

In [6]:
df.head()

,Unnamed: 0,trip_id,bike_id,day_of_week,start_hour,start_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,...,end_station_latitude,end_station_longitude,trip_duration,subscriber,birth_year,gender,trip_duration_minutes,age,Trip Range,age_category
0,0,LnQzQk,16013,Mon,18,2013-09-09 18:18:55,523,W 38 St & 8 Ave,40.754666,-73.991382,...,40.742388,-73.997262,993,Subscriber,1968,Female,16,45,long trip,Middle Age
1,1,IL9boN,15230,Thu,18,2013-09-12 18:38:53,257,Lispenard St & Broadway,40.719392,-74.002472,...,40.728419,-73.987140,581,Subscriber,1983,Male,9,30,average trip,Adult
2,2,46clGB,17942,Wed,19,2013-09-18 19:44:04,479,9 Ave & W 45 St,40.760193,-73.991255,...,40.768254,-73.988639,361,Subscriber,1989,Male,6,24,average trip,Young Adult
3,3,v7vdFt,19683,Sat,11,2013-09-28 11:54:37,527,E 33 St & 1 Ave,40.743156,-73.974347,...,40.756014,-73.967416,561,Subscriber,1988,Female,9,25,average trip,Young Adult
4,4,VGBsb5,18024,Sat,18,2013-09-07 18:08:22,521,8 Ave & W 31 St,40.750450,-73.994811,...,40.743943,-73.979661,2296,Non-Subscriber,1978,unknown,38,35,long trip,Adult


In [7]:
df.shape

(49977, 23)

In [8]:
df['station_count'] = df.groupby('start_station_name')['start_station_name'].transform('count')


In [9]:
df.head()

,Unnamed: 0,trip_id,bike_id,day_of_week,start_hour,start_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,...,end_station_longitude,trip_duration,subscriber,birth_year,gender,trip_duration_minutes,age,Trip Range,age_category,station_count
0,0,LnQzQk,16013,Mon,18,2013-09-09 18:18:55,523,W 38 St & 8 Ave,40.754666,-73.991382,...,-73.997262,993,Subscriber,1968,Female,16,45,long trip,Middle Age,228
1,1,IL9boN,15230,Thu,18,2013-09-12 18:38:53,257,Lispenard St & Broadway,40.719392,-74.002472,...,-73.987140,581,Subscriber,1983,Male,9,30,average trip,Adult,118
2,2,46clGB,17942,Wed,19,2013-09-18 19:44:04,479,9 Ave & W 45 St,40.760193,-73.991255,...,-73.988639,361,Subscriber,1989,Male,6,24,average trip,Young Adult,226
3,3,v7vdFt,19683,Sat,11,2013-09-28 11:54:37,527,E 33 St & 1 Ave,40.743156,-73.974347,...,-73.967416,561,Subscriber,1988,Female,9,25,average trip,Young Adult,207
4,4,VGBsb5,18024,Sat,18,2013-09-07 18:08:22,521,8 Ave & W 31 St,40.750450,-73.994811,...,-73.979661,2296,Non-Subscriber,1978,unknown,38,35,long trip,Adult,410


In [10]:
# import Nominatim API
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [11]:
geolocator = Nominatim(user_agent="bikeshare")
reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1, max_retries=0)

In [12]:
reverse("40.754666, -73.991382").raw['address']

{'amenity': 'Citi Bike - W 38 St & 8 Ave',
 'road': 'West 38th Street',
 'neighbourhood': 'Garment District',
 'borough': 'Manhattan',
 'county': 'New York County',
 'city': 'City of New York',
 'state': 'New York',
 'ISO3166-2-lvl4': 'US-NY',
 'postcode': '10018',
 'country': 'United States',
 'country_code': 'us'}

In [13]:
# create locations dataframe for map markers
locations = df[['start_station_id','start_station_name',\
'start_station_latitude','start_station_longitude', 'station_count']]\
.drop_duplicates(subset = ['start_station_id'])

In [14]:
# rename columns and reset index
locations.columns=['station id','station name','latitude','longitude', 'count']
locations.set_index('station id', inplace=True)
locations

,station name,latitude,longitude,count
station id,,,,
523,W 38 St & 8 Ave,40.754666,-73.991382,228
257,Lispenard St & Broadway,40.719392,-74.002472,118
479,9 Ave & W 45 St,40.760193,-73.991255,226
527,E 33 St & 1 Ave,40.743156,-73.974347,207
521,8 Ave & W 31 St,40.750450,-73.994811,410
...,...,...,...,...
431,Hanover Pl & Livingston St,40.688646,-73.982634,14
331,Pike St & Monroe St,40.711731,-73.991930,54
443,Bedford Ave & S 9th St,40.708531,-73.964090,20


In [15]:
# use API to get detailed location information for each station
locations_2=[]
for index, row in locations.iterrows():
    locations_2.append(reverse("{}, {}".format(row['latitude'],\
    row['longitude'])).raw['address'])
pd.DataFrame(locations_2[:10])

,amenity,road,neighbourhood,borough,county,city,state,ISO3166-2-lvl4,postcode,country,country_code,house_number,railway
0,Citi Bike - W 38 St & 8 Ave,West 38th Street,Garment District,Manhattan,New York County,City of New York,New York,US-NY,10018,United States,us,NaN,NaN
1,Citi Bike - Lispenard St & Broadway,Lispenard Street,Little Italy,Manhattan,New York County,City of New York,New York,US-NY,10013,United States,us,NaN,NaN
2,Citi Bike - 9th Avenue & West 45th Street,9th Avenue,Hell's Kitchen,Manhattan,New York County,City of New York,New York,US-NY,10019,United States,us,NaN,NaN
3,NaN,East 33rd Street,Kips Bay,Manhattan,New York County,City of New York,New York,US-NY,10016,United States,us,377,NaN
4,NaN,8th Avenue,Chelsea,Manhattan,New York County,City of New York,New York,US-NY,10001,United States,us,NaN,NaN
5,NaN,Astor Place,University Village,Manhattan,New York County,City of New York,New York,US-NY,10003,United States,us,NaN,Astor Place
6,Citi bike - W 4 St & 7 Ave S,West 4th Street,NaN,Manhattan,New York County,City of New York,New York,US-NY,10014,United States,us,NaN,NaN
7,Citi Bike - Bank St & Hudson St,Bank Street,NaN,Manhattan,New York County,City of New York,New York,US-NY,10014,United States,us,NaN,NaN
8,Citi Bike - West 51st Street & 6th Avenue,Rockefeller Plaza,Midtown East,Manhattan,New York County,City of New York,New York,US-NY,10111,United States,us,45,NaN
9,Citi Bike - West 24th Street & 7th Avenue,West 24th Street,Chelsea,Manhattan,New York County,City of New York,New York,US-NY,10011,United States,us,NaN,NaN


In [16]:
# create new stations dataframe
dfstations = pd.DataFrame(index=locations.index, data=locations_2,\
      columns=['neighbourhood','suburb','postcode'])
dfstations.rename(columns={"neighbourhood":"neighborhood",\
      "suburb": "borough", "postcode": "zipcode"}\
      ,inplace=True )

In [17]:
# shorten zip codes and change data types
dfstations['neighborhood'] = dfstations['neighborhood'].astype('category')
dfstations['borough'] = dfstations['borough'].astype('category')
dfstations['zipcode'] = dfstations['zipcode'].str[:5].astype('category')
cols = ['neighborhood', 'borough']

In [18]:
# add station name, count, latitude, and longitude to dataframe
dfstations[['station_name','latitude','longitude', 'count']] =\
 locations[['station name','latitude','longitude', 'count']]

In [19]:
dfstations.head(10)


,neighborhood,borough,zipcode,station_name,latitude,longitude,count
station id,,,,,,,
523,Garment District,NaN,10018,W 38 St & 8 Ave,40.754666,-73.991382,228
257,Little Italy,NaN,10013,Lispenard St & Broadway,40.719392,-74.002472,118
479,Hell's Kitchen,NaN,10019,9 Ave & W 45 St,40.760193,-73.991255,226
527,Kips Bay,NaN,10016,E 33 St & 1 Ave,40.743156,-73.974347,207
521,Chelsea,NaN,10001,8 Ave & W 31 St,40.750450,-73.994811,410
293,University Village,NaN,10003,Lafayette St & E 8 St,40.730287,-73.990765,505
380,NaN,NaN,10014,W 4 St & 7 Ave S,40.734011,-74.002939,386
346,NaN,NaN,10014,Bank St & Hudson St,40.736529,-74.006180,180
510,Midtown East,NaN,10111,W 51 St & 6 Ave,40.760660,-73.980420,171


In [20]:
# export data frame
dfstations.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'citibike_stations.csv'))

In [21]:
# Make an empty map
map = folium.Map(location=[40.712776, -74.005974], tiles="OpenStreetMap", zoom_start=12)

# Show the map
map

In [22]:
import math

# add marker one by one on the map, and account for Mercator deformation
for city in dfstations.itertuples():
   # local_deformation = math.cos(city.latitude * math.pi / 180)
    folium.Circle(
        location=[city.latitude, city.longitude],
        popup='%s (%.1f)' % (city.station_name, city.count),
       radius=city.count,
    ).add_to(map)

# Show the map again
map

In [23]:
# sort dfstations to find top 10 most frequented stations
dfstations2 = dfstations.sort_values(by='count', ascending=False)

In [24]:
df_first_10 = dfstations2.head(10)
print(df_first_10)

                           neighborhood borough zipcode  \
station id                                                
459         Manhattan Community Board 4     NaN   10011   
293                  University Village     NaN   10003   
519                       Midtown South     NaN   10178   
497                        Union Square     NaN   10010   
426                             Tribeca     NaN   10282   
521                             Chelsea     NaN   10001   
382                                 NaN     NaN   10003   
285                  University Village     NaN   10003   
380                                 NaN     NaN   10014   
477                    Theater District     NaN   10036   

                       station_name   latitude  longitude  count  
station id                                                        
459                W 20 St & 11 Ave  40.746745 -74.007756    536  
293           Lafayette St & E 8 St  40.730287 -73.990765    505  
519               Persh

In [25]:
# export first 10
df_first_10.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'top_10.csv'))

In [26]:
df_first_10.head()


,neighborhood,borough,zipcode,station_name,latitude,longitude,count
station id,,,,,,,
459,Manhattan Community Board 4,NaN,10011,W 20 St & 11 Ave,40.746745,-74.007756,536
293,University Village,NaN,10003,Lafayette St & E 8 St,40.730287,-73.990765,505
519,Midtown South,NaN,10178,Pershing Square N,40.751884,-73.977702,503
497,Union Square,NaN,10010,E 17 St & Broadway,40.737050,-73.990093,496
426,Tribeca,NaN,10282,West St & Chambers St,40.717548,-74.013221,465


In [27]:
df_first_10.shape

(10, 7)

In [28]:
# Create a Folium map centered around the mean of the coordinates
map = folium.Map(location=[df_first_10['latitude'].mean(), df_first_10['longitude'].mean()], zoom_start=12)
map

In [29]:
# Add CircleMarkers for each city
for index, row in df_first_10.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=row['count'] / 10,  # Adjust the divisor to control the bubble size
        popup=row['station_name'],
        color='blue',
        fill=True,
        fill_color='blue'
    ).add_to(map)

# Show the map
map

In [30]:
# sort dfstations to find 10 least frequented stations
dfstations3 = dfstations.sort_values(by='count', ascending=True)
print(dfstations3)

                           neighborhood       borough zipcode  \
station id                                                      
2001                                NaN           NaN   11251   
2005                                NaN           NaN   11251   
372                                 NaN  Clinton Hill   11225   
321                               Dumbo           NaN   11201   
119                         Fort Greene           NaN   11251   
...                                 ...           ...     ...   
426                             Tribeca           NaN   10282   
497                        Union Square           NaN   10010   
519                       Midtown South           NaN   10178   
293                  University Village           NaN   10003   
459         Manhattan Community Board 4           NaN   10011   

                             station_name   latitude  longitude  count  
station id                                                              
2001    

In [31]:
df_last_10 = dfstations3.head(10)
print(df_last_10)

                  neighborhood       borough zipcode  \
station id                                             
2001                       NaN           NaN   11251   
2005                       NaN           NaN   11251   
372                        NaN  Clinton Hill   11225   
321                      Dumbo           NaN   11201   
119                Fort Greene           NaN   11251   
421                Fort Greene           NaN   11205   
431          Downtown Brooklyn           NaN   11201   
298          Downtown Brooklyn           NaN   11227   
436                        NaN  Clinton Hill   11233   
343         Brooklyn Navy Yard           NaN   11205   

                             station_name   latitude  longitude  count  
station id                                                              
2001                  7 Ave & Farragut St  40.698921 -73.973330      5  
2005               Railroad Ave & Kay Ave  40.705312 -73.971001      5  
372             Franklin Ave & Myrt

In [32]:
# export last 10
df_last_10.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'bottom_10.csv'))

In [33]:
df_last_10.head()

,neighborhood,borough,zipcode,station_name,latitude,longitude,count
station id,,,,,,,
2001,NaN,NaN,11251,7 Ave & Farragut St,40.698921,-73.973330,5
2005,NaN,NaN,11251,Railroad Ave & Kay Ave,40.705312,-73.971001,5
372,NaN,Clinton Hill,11225,Franklin Ave & Myrtle Ave,40.694528,-73.958089,8
321,Dumbo,NaN,11201,Cadman Plaza E & Red Cross Pl,40.699918,-73.989718,11
119,Fort Greene,NaN,11251,Park Ave & St Edwards St,40.696089,-73.978034,11


In [34]:
df_last_10.shape

(10, 7)

In [35]:
# Create a Folium map centered around the mean of the coordinates
map = folium.Map(location=[df_last_10['latitude'].mean(), df_last_10['longitude'].mean()], zoom_start=12)
map

In [36]:
# Add CircleMarkers for each city
for index, row in df_last_10.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=row['count'] / 3,  # Adjust the divisor to control the bubble size
        popup=row['station_name'],
        color='blue',
        fill=True,
        fill_color='blue'
    ).add_to(map)

# Show the map
map